In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)
#Read the sonar dataset
df = pd.read_csv('YearPredictionMSD.txt', header=None)
df

In [ ]:
print(len(df.columns))
# X = df[df.columns[1:]].values
# 1 and 2 are skewed data
X = df[set(df.columns) - set(df.columns[0:1])].values # - set(df.columns[21:25]) - set(df.columns[53:54])].values
       
Y=df[df.columns[0]]

In [ ]:
# y_t = tf.one_hot(tf.constant(Y[:1000]-1930), depth=89)
# for i in range(len(y_t)):
#     print(Y[i],y_t[i])

In [ ]:
#Transform the data in training and testing
# train_x=X[:1000]
# test_x=X[-200:]
# train_y = tf.one_hot(tf.constant(Y[:1000]-1930), depth=89)
# test_y=tf.one_hot(tf.constant(Y[-200:]-1930), depth=89)
train_x=X[:463715]
test_x=X[463716:]
train_y = tf.one_hot(tf.constant(Y[:463715]-1930), depth=89)
test_y=tf.one_hot(tf.constant(Y[463716:]-1930), depth=89)

In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_x))
print(normalizer.mean.numpy())

In [ ]:
first = np.array(train_x[:1])

with np.printoptions(precision=2, suppress=True):
    print('First example:', first)
    print()
    print('Normalized:', normalizer(first).numpy())

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=89)
])

In [ ]:
linear_model.predict(train_x[:10])

In [ ]:
linear_model.layers[1].kernel

In [ ]:
learning_rate=0.1
epoch=10
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
    loss='mean_absolute_error')
# mean_squared_error

In [ ]:
history = linear_model.fit(
    train_x, train_y, 
    epochs=epoch,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
linear_model.evaluate(test_x, test_y, verbose=0)

In [ ]:
linear_model.metrics_names

In [ ]:
test_predictions = dnn_model.predict(test_x)
error = test_predictions - test_y
error

In [ ]:
sum(sum(np.square(error)))/2/len(test_y)